In [11]:
### 구분복잡성 지수

from konlpy.tag import Kkma
from konlpy.tag import Komoran

# KoNLPy의 형태소 분석기 로드
kkma = Kkma()
komoran = Komoran()

def syntactic_complexity_korean(text):
    sentences = kkma.sentences(text)  # 텍스트를 문장으로 분리합니다.
    num_sentences = len(sentences)
    
    num_clauses = 0
    for sentence in sentences:
        morphemes = komoran.pos(sentence)  # 문장을 형태소 분석합니다.
        num_clauses += sum(1 for word, tag in morphemes if tag in ['VV', 'VA'])  # 동사나 형용사를 종속 절로 간주합니다.
    
    return num_clauses / num_sentences if num_sentences > 0 else 0


original_text = '''【판시사항】
[1] 토지임차인의 지상물매수청구권을 배제하는 약정의 효력
[2] 임차기간이 만료되기 전에 임차권자가 제3자에게 임차기간의 만료를 조건으로 지상물매수청구권을 양도한 경우, 임차기간 만료 후 제3자가 토지 소유자에 대하여 직접 지상물매수청구권을 행사하기 위한 요건【판결요지】
[1]건물의 소유를 목적으로 한 토지의 임차인이 임대차계약을 체결할 당시 임대인과 사이에 건물 기타 지상시설 일체를 포기하기로 약정하였다 하더라도 임대차계약의 조건이나 계약이 체결된 경위 등 제반사정을 종합적으로 고려하여 실질적으로 임차인에게 불리하다고 볼 수 없는 특별한 사정이 인정되지 않는 한 위와 같은 약정은 민법 제643조소정의 토지임차인의 지상물매수청구권을 배제하기로 하는 약정으로서 임차인에게 불리한 것이므로 민법 제652조의 규정에 위반되어 그 효력이 없다.'''
simplified_text = "[1] 땅을 빌린 사람이 건물을 짓기 위해 땅을 빌릴 때, 땅 주인과 건물이나 다른 시설을 포기하기로 약속했더라도, 그 약속이 땅을 빌린 사람에게 불리하지 않다는 특별한 이유가 없다면, 그 약속은 법에 어긋나서 효력이 없다. [2] 만약 땅을 빌린 사람이 임대 기간이 끝나기 전에 다른 사람에게 그 권리를 넘겼다면, 임대 기간이 끝난 후 그 다른 사람이 땅 주인에게 직접 물건을 사달라고 요구하기 위해서는 특별한 조건이 필요하다."

original_syntactic_complexity = syntactic_complexity_korean(original_text)
simplified_syntactic_complexity = syntactic_complexity_korean(simplified_text)

print(f"Original syntactic complexity: {original_syntactic_complexity}, Simplified: {simplified_syntactic_complexity}")


Original syntactic complexity: 9.0, Simplified: 7.0


In [17]:
import math
import re
from konlpy.tag import Okt

text1 = """
【판시사항】
[1] 토지임차인의 지상물매수청구권을 배제하는 약정의 효력
[2] 임차기간이 만료되기 전에 임차권자가 제3자에게 임차기간의 만료를 조건으로 지상물매수청구권을 양도한 경우, 임차기간 만료 후 제3자가 토지 소유자에 대하여 직접 지상물매수청구권을 행사하기 위한 요건【판결요지】
[1]건물의 소유를 목적으로 한 토지의 임차인이 임대차계약을 체결할 당시 임대인과 사이에 건물 기타 지상시설 일체를 포기하기로 약정하였다 하더라도 임대차계약의 조건이나 계약이 체결된 경위 등 제반사정을 종합적으로 고려하여 실질적으로 임차인에게 불리하다고 볼 수 없는 특별한 사정이 인정되지 않는 한 위와 같은 약정은 민법 제643조소정의 토지임차인의 지상물매수청구권을 배제하기로 하는 약정으로서 임차인에게 불리한 것이므로 민법 제652조의 규정에 위반되어 그 효력이 없다."""
text2 = """
[1] 땅을 빌린 사람이 건물을 짓기 위해 땅을 빌릴 때, 땅 주인과 건물이나 다른 시설을 포기하기로 약속했더라도, 그 약속이 땅을 빌린 사람에게 불리하지 않다는 특별한 이유가 없다면, 그 약속은 법에 어긋나서 효력이 없다. [2] 만약 땅을 빌린 사람이 임대 기간이 끝나기 전에 다른 사람에게 그 권리를 넘겼다면, 임대 기간이 끝난 후 그 다른 사람이 땅 주인에게 직접 물건을 사달라고 요구하기 위해서는 특별한 조건이 필요하다."""

def calculate_fog_and_length(text):
    # 한국어 형태소 분석기
    okt = Okt()

    # 텍스트를 문장 단위로 분리
    sentences = re.split(r'[.!?]', text)

    # 문장당 평균 단어 수 계산 (ASL)
    total_words = 0
    total_sentences = len(sentences)
    for sentence in sentences:
        words = okt.morphs(sentence.strip())
        total_words += len(words)

    ASL = total_words / total_sentences if total_sentences > 0 else 0

    # 복잡한 단어 비율 계산 (3음절 이상의 단어)
    complex_words = 0
    for word in okt.morphs(text):
        if len(word) >= 3:
            complex_words += 1

    percentage_of_complex_words = (complex_words / total_words) * 100 if total_words > 0 else 0

    # FOG 계산
    FOG = (ASL + percentage_of_complex_words) * 0.4

    # LENGTH 계산
    Word_num = total_words
    LENGTH = math.log(Word_num) if Word_num > 0 else 0

    return ASL, percentage_of_complex_words, FOG, LENGTH

ASL1, complex_words1, FOG1, LENGTH1 = calculate_fog_and_length(text1)
print(f"텍스트 1:")
print(f"ASL (문장당 평균 단어 수): {ASL1}")
print(f"복잡한 단어 비율: {complex_words1}%")
print(f"Gunning Fog Index (FOG): {FOG1}")
print(f"LENGTH: {LENGTH1}")

ASL2, complex_words2, FOG2, LENGTH2 = calculate_fog_and_length(text2)
print(f"\n텍스트 2:")
print(f"ASL (문장당 평균 단어 수): {ASL2}")
print(f"복잡한 단어 비율: {complex_words2}%")
print(f"Gunning Fog Index (FOG): {FOG2}")
print(f"LENGTH: {LENGTH2}")


텍스트 1:
ASL (문장당 평균 단어 수): 92.5
복잡한 단어 비율: 11.351351351351353%
Gunning Fog Index (FOG): 41.54054054054055
LENGTH: 5.220355825078324

텍스트 2:
ASL (문장당 평균 단어 수): 34.333333333333336
복잡한 단어 비율: 11.650485436893204%
Gunning Fog Index (FOG): 18.393527508090617
LENGTH: 4.634728988229636
